# Interactive LLM Agent with User Feedback

FLAML offers an experimental feature of interactive LLM agents, which can be used to solve various tasks, including coding and math problem-solving.

In this notebook, we demonstrate how to use `PythonAgent` and `UserProxyAgent` to solve a challenging math problem. Here `PythonAgent` is an LLM-based agent that can write Python code (in a Python coding block) for a user to execute for a given task. `UserProxyAgent` is an agent which serves as a proxy for a user to execute the code written by `PythonAgent`. By setting `user_interaction_mode` properly, the `UserProxyAgent` can also prompt the user for feedback to `PythonAgent`. For example, when `user_interaction_mode` is set to "ALWAYS", the `UserProxyAgent` will always prompt the user for feedback. When user feedback is provided, the `UserProxyAgent` will directly pass the feedback to `PythonAgent` without doing any additional steps. When no user feedback is provided, the `UserProxyAgent` will execute the code written by `PythonAgent` directly and returns the execution results (success or failure and corresponding outputs) to `PythonAgent`.

## Requirements

FLAML requires `Python>=3.7`. To run this notebook example, please install flaml with the [openai] option:
```bash
pip install flaml[openai]==1.2.4
```

In [1]:
# %pip install flaml[openai]==1.2.4

## Set your API Endpoint

The [`config_list_gpt4_gpt35`](https://microsoft.github.io/FLAML/docs/reference/autogen/oai/openai_utils#config_list_gpt4_gpt35) function tries to create a list of gpt-4 and gpt-3.5 configurations using Azure OpenAI endpoints and OpenAI endpoints. It assumes the api keys and api bases are stored in the corresponding environment variables or local txt files:

- OpenAI API key: os.environ["OPENAI_API_KEY"] or `openai_api_key_file="key_openai.txt"`.
- Azure OpenAI API key: os.environ["AZURE_OPENAI_API_KEY"] or `aoai_api_key_file="key_aoai.txt"`. Multiple keys can be stored, one per line.
- Azure OpenAI API base: os.environ["AZURE_OPENAI_API_BASE"] or `aoai_api_base_file="base_aoai.txt"`. Multiple bases can be stored, one per line.

It's OK to have only the OpenAI API key, or only the Azure Open API key + base.


In [2]:
from flaml import oai

config_list = oai.config_list_gpt4_gpt35()

## Construct Agents

We construct the python agent and the user proxy agent.

In [ ]:
from flaml.autogen.agent.coding_agent import PythonAgent
from flaml.autogen.agent.user_proxy_agent import UserProxyAgent

# create an assistant which is essentially a PythonAgent instance named "coding_agent"
assistant = PythonAgent(name="coding_agent", request_timeout=600, seed=42, config_list=config_list)
# create a UserProxyAgent instance named "user"
user = UserProxyAgent(
    name="user",
    human_input_mode="ALWAYS",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.rstrip().endswith("TERMINATE"),
)

# the purpose of the following two lines is to log the conversation history
conversations = {}
oai.ChatCompletion.start_logging(conversations)


## Perform a task

We invoke the `receive()` method of the coding agent to start the conversation. When you run the cell below, you will be prompted to provide feedback after receving a message from the coding agent. If you don't provide any feedback (by pressing Enter directly), the user proxy agent will try to execute the code suggested by the coding agent on behalf of you, or terminate if the coding agent sends a "TERMINATE" signal in the end of the message.

In [6]:
math_problem_to_solve = """
     Find $a + b + c$, given that $x+y \\neq -1$ and 
     \\begin{align}
	ax + by + c & = x + 7,\\
	a + bx + cy & = 2x + 6y,\\
	ay + b + cx & = 4x + y.
	\\end{align}.
    """

# the assistant receives a message from the user, which contains the task description
assistant.receive(math_problem_to_solve, user)


**** coding_agent received message from user ****


     Find $a + b + c$, given that $x+y \neq -1$ and 
     \begin{align}
	ax + by + c & = x + 7,\
	a + bx + cy & = 2x + 6y,\
	ay + b + cx & = 4x + y.
	\end{align}.
    

**** user received message from coding_agent ****

Let's solve this system of linear equations using Python. We will use the numpy library to solve the equations.

```python
import numpy as np

# Define the coefficients matrix and the constant terms vector
A = np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]])
B = np.array([1 + 7, 2 + 6, 4 + 1])

# Solve the linear system
x = np.linalg.solve(A, B)

# Compute the sum of the solution components
result = np.sum(x)

# Print the result
print(result)
```

Save this code in a Python file, e.g., `linear_solver.py` and execute it. Let me know the output.

**** coding_agent received message from user ****

Are you assuming x=y=1? How about assuming x=y=0?

**** user received message from coding_agent ****

I apologize for the confusio

The human user provided feedback such as "Are you assuming x=y=1? How about assuming x=y=0?", "what's the value if x=y=0?" At some steps, the human user didn't provide feedback, and the code was executed. The executed results and error messages are returned to the coding agent, and the coding agent was able to modify the code to rectify. In the end, the task is complet and a "TERMINATE" signal is sent from the coding agent. The user skipped feedback in the end and the conversation is finished.
After the conversation is finished, we can inspect the log of the conversation between the two agents.

In [7]:
print(conversations)

{'[{"content": "You suggest python code (in a python coding block) for a user to execute for a given task. If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Finish the task smartly. Don\'t suggest shell command. Don\'t include multiple code blocks in one response. Use \'print\' function for the output when relevant. Check the execution result returned by the user.\\n    If the result indicates there is an error, fix the error and output the code again.\\n    Reply \\"TERMINATE\\" in the end when the task is done.\\n    ", "role": "system"}, {"content": "\\n     Find $a + b + c$, given that $x+y \\\\neq -1$ and \\n     \\\\begin{align}\\n\\tax + by + c & = x + 7,\\\\\\n\\ta + bx + cy & = 2x + 6y,\\\\\\n\\tay + b + cx & = 4x + y.\\n\\t\\\\end{align}.\\n    ", "role": "user"}, {"content": "Let\'s solve this system of linear equations using Python. We will use the numpy library to solve the equations.\\n